## Photometry

In [1]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from hostphot import (coadd, image_masking, 
                      local_photometry as lp,
                      global_photometry as gp)
from hostphot.utils import get_survey_filters

import warnings
# ignore annoying warnings
warnings.filterwarnings("ignore")

coadd_dict = {'DES':'riz', 'PS1':'riz', 'GALEX':['FUV', 'NUV'], '2MASS':['H', 'J', 'Ks'], 'unWISE':["W1", "W2"], 'SDSS':'riz'}
AP_RADII = [1, 2, 3, 4]  # aperture radii for local photometry

In [2]:
import time

def log(text):
    with open("logfile.txt", "a") as f:
        f.write(f"[{time.strftime('%H:%M:%S')}] {text}\n")

In [3]:
import winsound
import requests

def make_noise():
    duration = 1000  # milliseconds
    freq = 440  # Hz
    winsound.Beep(freq, duration)

webhookurl = r'https://discord.com/api/webhooks/xxxxx'

def notify(message = 'done!'):
    requests.post(webhookurl, json={"content": "<@xxxxx> " + message})
    make_noise()

In [4]:
# DES + low-z samples
full_df = pd.read_csv('04-BBCFITS/SALT2mu_DES+LOWZ_G10.FITRES', 
                 delim_whitespace=True, skiprows=7, comment='#')

lowz_df = full_df[full_df.IDSURVEY.values!=10]  # low-z only

# Host info from the Open Supernova Catalog for the low-z sample
osc_df = pd.read_csv('osc_results.csv')
osc_df.rename({'SN_Name':'CID'}, axis=1, inplace=True)
osc_df.CID.replace('SN', '', inplace=True, regex=True)

# merge both dataframes
lowz_df = pd.merge(lowz_df, osc_df, on='CID', how='outer')
lowz_df

,VARNAMES:,CID,CIDint,IDSURVEY,TYPE,FIELD,CUTFLAG_SNANA,zHEL,zHELERR,zCMB,...,biasCor_mu,biasCorErr_mu,biasCor_mB,biasCor_x1,biasCor_c,biasScale_muCOV,IDSAMPLE,Host_Name,Host_RA,Host_DEC
0,SN:,2004ef,1,64,0,NaN,1,0.03098,0.0,0.02979,...,-0.032,0.003,-0.008,-0.014,0.008,0.956,0,UGC 12158,340.5442,19.9969
1,SN:,2006et,2,64,0,NaN,1,0.02258,0.0,0.02160,...,-0.025,0.004,-0.005,0.015,0.007,0.959,0,NGC 232,10.6875,-23.5611
2,SN:,2006ev,3,64,0,NaN,1,0.02873,0.0,0.02762,...,-0.030,0.003,-0.007,-0.009,0.007,0.957,0,UGC 11758,322.7417,13.9858
3,SN:,2006ef,4,64,0,NaN,1,0.01768,0.0,0.01682,...,-0.010,0.002,-0.012,-0.007,-0.001,0.951,0,NGC 809,31.0792,-8.7356
4,SN:,2005lu,5,64,0,NaN,1,0.03224,0.0,0.03154,...,-0.031,0.007,-0.006,0.037,0.010,0.956,0,MCG -03-07-40,39.0167,-17.2639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,SN:,2006eq,124,5,1,NaN,1,0.04945,0.0,0.04834,...,-0.070,0.007,-0.012,-0.095,0.015,0.901,0,SDSS J12837.60+011348.6,322.1567,1.2302
118,SN:,2006lu,125,5,1,NaN,1,0.05446,0.0,0.05549,...,0.032,0.005,-0.052,-0.015,-0.028,0.844,0,A091517-2536,138.8208,-25.6000
119,SN:,2006py,126,5,1,NaN,1,0.05783,0.0,0.05661,...,-0.045,0.005,-0.019,-0.030,0.007,0.836,0,SDSS J24142.04-000812.9,340.4250,-0.1369
120,SN:,2008bq,128,5,1,NaN,1,0.03314,0.0,0.03360,...,-0.027,0.003,-0.010,-0.008,0.005,0.947,0,ESO 308-G25,100.2625,-38.0356


In [5]:
def create_results_dict(filters, ap_radii):
    """Creates a dictionary to save the local photometry."""
    results_dict = {}    
    # iterate for different apertures
    for ap in ap_radii:
        mags_dict = {f'{filt}{ap}':[] for filt in filters}
        mags_err_dict = {f'{filt}{ap}_err':[] for filt in filters}
        results_dict.update(mags_dict)
        results_dict.update(mags_err_dict)
    
    return results_dict

In [8]:
surveys = ['DES', 'PS1', 'GALEX', '2MASS', 'unWISE', 'SDSS']
redo = True
progress_file = 'progress.txt'
resume_progress = True

if resume_progress:
    with open(progress_file) as f:
        lines = f.read().splitlines()
        sn_curr = lines[0]
        survey_curr = lines[1]
    sn_checkpoint_found = False
    survey_checkpoint_found = False

# photometry
for name, z, ra, dec in zip(lowz_df.CID, lowz_df.zCMB, 
                            lowz_df.RA, lowz_df.DECL):
    if resume_progress and not sn_checkpoint_found:
        if name == sn_curr:
            sn_checkpoint_found = True
        else:
            continue
    log(f"Starting: {name}")
    for survey in surveys:
        if resume_progress and not survey_checkpoint_found:
            if survey == survey_curr:
                survey_checkpoint_found = True
            else:
                continue
        local_file = os.path.join('images', name, 
                                f'local_phot_{survey}.csv')
        if not redo and os.path.isfile(local_file):
            continue
        
        log(f"{survey}")
        with open(progress_file, 'w') as f:
            f.write(f'{name}\n{survey}')
        filters = get_survey_filters(survey)
        
        # create dictionary to store the results
        results_dict = create_results_dict(filters, AP_RADII)
        results_dict['name'] = [name]
        results_dict['ra'] = [ra]
        results_dict['dec'] = [dec]
        results_dict['zspec'] = [z]

        for filt in filters:
            null_filter = False
            if f"masked_{survey}_{filt}.fits" in os.listdir(f"images/{name}"):
                try:
                    mags, mags_err, *_ = lp.photometry(name, ra, dec, z, filt, 
                                                survey, ap_radii=AP_RADII)
                    for i, ap in enumerate(AP_RADII):
                        results_dict[f'{filt}{ap}'].append(mags[i])
                        results_dict[f'{filt}{ap}_err'].append(mags_err[i])
                except Exception as e:
                    log(str(e))
                    null_filter = True
            else:
                null_filter = True
            if null_filter:
                for i, ap in enumerate(AP_RADII):
                    results_dict[f'{filt}{ap}'].append(None)
                    results_dict[f'{filt}{ap}_err'].append(None)

        sn_phot = pd.DataFrame(results_dict)
        sn_phot.to_csv(local_file, index=False)

In [ ]:
notify("local done")

In [6]:
def create_results_dict_global(filters):
    """Creates a dictionary to save the global photometry."""
    results_dict = {filt:[] for filt in filters}
    
    mags_err_dict = {f'{filt}_err':[] for filt in filters}
    results_dict.update(mags_err_dict)
    
    return results_dict

In [7]:
surveys = ['DES', 'PS1', 'GALEX', '2MASS', 'unWISE', 'SDSS']
eps = 0.00005
threshold = 6
redo = True
progress_file = 'glob_progress.txt'
resume_progress = True

if resume_progress:
    with open(progress_file) as f:
        lines = f.read().splitlines()
        sn_curr = lines[0]
        survey_curr = lines[1]
    sn_checkpoint_found = False
    survey_checkpoint_found = False
    
for name, z, ra, dec, host_ra, host_dec in zip(lowz_df.CID, lowz_df.zCMB, 
                                                lowz_df.RA, lowz_df.DECL,
                                              lowz_df.Host_RA, lowz_df.Host_DEC):
    if resume_progress and not sn_checkpoint_found:
        if name == sn_curr:
            sn_checkpoint_found = True
        else:
            continue
    log(f"Starting: {name}")
    for survey in surveys:
        if resume_progress and not survey_checkpoint_found:
            if survey == survey_curr:
                survey_checkpoint_found = True
            else:
                continue
        filters = get_survey_filters(survey)
        global_file = os.path.join('images', name, 
                                f'global_phot_{survey}.csv')
        if not redo and os.path.isfile(global_file):
            continue
        
        log(f"{survey}")
        with open(progress_file, 'w') as f:
            f.write(f'{name}\n{survey}')
        
        # create dictionary to store the results
        results_dict = create_results_dict_global(filters)
        results_dict['name'] = [name]
        results_dict['ra'] = [ra]
        results_dict['dec'] = [dec]
        results_dict['host_ra'] = [host_ra]
        results_dict['host_dec'] = [host_dec]
        results_dict['zspec'] = [z]
        
        try:
            # get common aperture parameters
            aperture_params = gp.extract_kronparams(name, host_ra, host_dec, 
                                                    coadd_dict[survey], survey, eps=eps,
                                                    threshold=threshold,
                                                ra=ra, dec=dec,)
            for filt in filters:
                mag, mag_err, *_ = gp.photometry(name, host_ra, host_dec, filt, survey, 
                                            ra=ra, dec=dec,
                                            aperture_params=aperture_params, eps=eps)
                results_dict[filt].append(mag)
                results_dict[f'{filt}_err'].append(mag_err)
        except FileNotFoundError as e:
            log(str(e))
            log('Skipping: ' + survey)
            continue
        except Exception as e:
            log(str(e))
            log('Skipping: ' + survey)
            continue
                
        sn_phot = pd.DataFrame(results_dict)
        sn_phot.to_csv(global_file, index=False)